In [10]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage import color
from sklearn.metrics import jaccard_score 

In [11]:
# Derivatives matrix and energy matrix functions
def derivative_matrix(n):
    D = -2 * np.eye(n)
    for i in range(n - 1):
        D[i, i + 1] = 1
        D[i + 1, i] = 1
    D[0, -1] = 1
    D[-1, 0] = 1
    return D

In [12]:
def internal_energy_matrix(n, alpha, beta, gamma):
    D = derivative_matrix(n)
    A = alpha * np.dot(D, D) + beta * np.dot(D, np.dot(D, D))
    return np.linalg.inv(np.eye(n) - gamma * A)

# External energy (image gradients)
def external_energy(image):
    grad_y, grad_x = np.gradient(image)
    f_ext_x = -grad_x
    f_ext_y = -grad_y
    return f_ext_x, f_ext_y

In [13]:
# Custom interpolation function for external forces
def interp2(v, xq, yq):
    x = np.floor(xq).astype(int)
    y = np.floor(yq).astype(int)
    x1 = np.clip(x, 0, v.shape[1] - 1)
    x2 = np.clip(x + 1, 0, v.shape[1] - 1)
    y1 = np.clip(y, 0, v.shape[0] - 1)
    y2 = np.clip(y + 1, 0, v.shape[0] - 1)
    Q11 = v[y1, x1]
    Q12 = v[y1, x2]
    Q21 = v[y2, x1]
    Q22 = v[y2, x2]
    xq = xq - x1
    yq = yq - y1
    return (Q11 * (1 - xq) * (1 - yq) +
            Q12 * xq * (1 - yq) +
            Q21 * (1 - xq) * yq +
            Q22 * xq * yq)

In [14]:
# Evolve the snake over iterations
def evolve_snake(snake, img_smooth, alpha, beta, gamma, iterations):
    n = len(snake)
    A_inv = internal_energy_matrix(n, alpha, beta, gamma)
    f_ext_x, f_ext_y = external_energy(img_smooth)
    img_h, img_w = img_smooth.shape

    for _ in range(iterations):
        snake[:, 0] = np.clip(snake[:, 0], 0, img_h - 1)
        snake[:, 1] = np.clip(snake[:, 1], 0, img_w - 1)
        fx = interp2(f_ext_x, snake[:, 1], snake[:, 0])
        fy = interp2(f_ext_y, snake[:, 1], snake[:, 0])
        snake[:, 0] = np.dot(A_inv, snake[:, 0] + gamma * fy)
        snake[:, 1] = np.dot(A_inv, snake[:, 1] + gamma * fx)

    return snake

In [15]:
# Show histograms and determine channel with lowest mean
def display_histograms(image_rgb, contour_mask):
    color_channels = ('Red', 'Green', 'Blue')
    histograms = [cv2.calcHist([image_rgb], [i], contour_mask, [256], [0, 256]) for i in range(3)]
    means = [np.mean(hist) for hist in histograms]

    # plt.figure(figsize=(12, 4))
    # for i, hist in enumerate(histograms):
    #     plt.subplot(1, 3, i + 1)
    #     plt.plot(hist, color=color_channels[i].lower())
    #     plt.title(f'Histogram for {color_channels[i]} Channel')
    #     plt.xlabel('Pixel Intensity')
    #     plt.ylabel('Frequency')
    # plt.show()

    min_channel = np.argmin(means)
    return min_channel, np.mean(image_rgb[:, :, min_channel][contour_mask > 0])

In [16]:
# Calculate IoU score
def iou_score(mask_true, mask_pred):
    mask_true_flat = mask_true.flatten()
    mask_pred_flat = mask_pred.flatten()
    return jaccard_score(mask_true_flat, mask_pred_flat, average='binary')

In [17]:
# Process video with active contour and region growing
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Unable to open video")
        exit()

    width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = 60
    out = cv2.VideoWriter('result (no library).mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    alpha, beta, gamma, iterations = 0.1, 10, 0.1, 100
    ret, frame = cap.read()
    if not ret:
        print("Failed to read video")
        return

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    gray_smooth = cv2.GaussianBlur(color.rgb2gray(frame_rgb), (5, 5), 1)
    h, w = gray_smooth.shape
    s = np.linspace(0, 2 * np.pi, 200)
    r, c = h // 2 + 100 * np.sin(s), w // 2 + 100 * np.cos(s)
    snake = np.array([r, c]).T

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        gray_smooth = cv2.GaussianBlur(color.rgb2gray(frame_rgb), (5, 5), 1)

        # Evolve the snake in the current frame
        snake = evolve_snake(snake, gray_smooth, alpha, beta, gamma, iterations)

        # Create mask for the contour
        mask_contour = np.zeros_like(gray_smooth, dtype=np.uint8)
        cv2.polylines(mask_contour, [np.int32(snake)], isClosed=True, color=1, thickness=1)

        # Display histograms and determine the channel with the lowest mean
        min_channel, mean_intensity = display_histograms(frame_rgb, mask_contour)
        threshold = mean_intensity

        # Apply region growing based on the mean intensity
        image_channel = frame_rgb[:, :, min_channel].astype(np.uint8)
        mask = np.zeros((image_channel.shape[0] + 2, image_channel.shape[1] + 2), np.uint8)
        seed_point = (frame_rgb.shape[1] // 2, frame_rgb.shape[0] // 2)
        cv2.floodFill(image_channel, mask, seed_point, 255, (threshold,) * 3, (threshold,) * 3, cv2.FLOODFILL_MASK_ONLY)
        region_mask = mask[1:-1, 1:-1]

        # IoU calculation between active contour and region-growing masks
        iou = iou_score(mask_contour > 0, region_mask > 0)
        print(f"IoU score for frame: {iou:.4f}")

        # Combine contour with original frame
        frame_with_contour = frame.copy()
        for point in snake:
            x, y = int(point[1]), int(point[0])
            cv2.circle(frame_with_contour, (x, y), 2, (0, 255, 0), -1)

        # Draw the region growing result
        frame_with_contour[region_mask == 255] = (255, 0, 0)  # Color the region in red

        out.write(frame_with_contour)
        cv2.imshow('Contour Video', frame_with_contour)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [18]:
# Run the video processing
video_path = 'test.mp4'
process_video(video_path)

IoU score for frame: 0.0027
IoU score for frame: 0.0027
IoU score for frame: 0.0027
IoU score for frame: 0.0027
IoU score for frame: 0.0027
IoU score for frame: 0.0027
IoU score for frame: 0.0027
IoU score for frame: 0.0026
IoU score for frame: 0.0027
IoU score for frame: 0.0026
IoU score for frame: 0.0027
IoU score for frame: 0.0027
IoU score for frame: 0.0027
IoU score for frame: 0.0026
IoU score for frame: 0.0026
IoU score for frame: 0.0027
IoU score for frame: 0.0026
IoU score for frame: 0.0026
IoU score for frame: 0.0027
IoU score for frame: 0.0027
IoU score for frame: 0.0027
IoU score for frame: 0.0027
IoU score for frame: 0.0027
IoU score for frame: 0.0027
IoU score for frame: 0.0027
IoU score for frame: 0.0027
IoU score for frame: 0.0027
IoU score for frame: 0.0027
IoU score for frame: 0.0027
IoU score for frame: 0.0027
IoU score for frame: 0.0027
IoU score for frame: 0.0027
IoU score for frame: 0.0027
IoU score for frame: 0.0027
IoU score for frame: 0.0027
IoU score for frame: